In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/Git/tinympc/wrappers/TinyMPC-julia/codegen_examples/Project.toml`


In [2]:
using RobotZoo:Quadrotor
using RobotDynamics
import ForwardDiff as FD
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

import MeshCat as mc
using ColorTypes
using GeometryBasics: HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

include(joinpath(@__DIR__,"utils/cartpole_animation.jl"))

using Plots
# using Printf

In [3]:
# Define ground truth cartpole ODE for simulation
function dynamics(params::NamedTuple, x::Vector, u)
    # cartpole physical parameters 
    mc, mp, l = params.mc, params.mp, params.l
    g = 9.81
    
    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])

    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    return [qd;qdd]

end

function rk4(params::NamedTuple, x::Vector,u,dt::Float64)
    k1 = dt*dynamics(params, x, u)
    k2 = dt*dynamics(params, x + k1/2, u)
    k3 = dt*dynamics(params, x + k2/2, u)
    k4 = dt*dynamics(params, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

rk4 (generic function with 1 method)

In [59]:
# Define problem parameters
params = (mc = 1.2, mp = 0.16, l = 0.55)

# states and control sizes 
nx = 4 
nu = 1 

# desired x and u
xgoal = [0, pi, 0, 0]
ugoal = [0]

# initial condition
x0 = xgoal + [0, 0, 0, 0]

# simulation size 
dt = 0.01
tf = 10.0
t_vec = 0:dt:tf
Nsim = length(t_vec)+100
Nh = 10 # horizon length

# define cost matrices
Q = diagm([1,1,0.05,1])
R = [0.1]
R = reshape(R,(nu,nu))
ρ = 0.1

# Determine linearized dynamics of the cartpole about its upright position
A = FD.jacobian(dx -> rk4(params, dx, ugoal, dt), xgoal)
B = FD.jacobian(du -> rk4(params, xgoal, du, dt), ugoal)

xmin = -5 .* ones((nx,Nh))
xmax = 5 .* ones((nx,Nh))
umin = -5 .* ones((nu,Nh))
umax = 5 .* ones((nu,Nh))

# Solver options
abs_pri_tol = 1e-3
rel_pri_tol = 1e-3
max_iter = 100
verbose = 1

tinympc_dir = "/home/sam/Git/tinympc/TinyMPC/"
codegen_dir = "/generated_code"

"/generated_code"

In [27]:
# Create wrapper to generate TinyMPC code
using Libdl

# create pointer to shared library
TinyMPC_lib = dlopen(tinympc_dir * "/build/src/tinympc/libtinympc.so")

# create pointer to tiny_codegen function
tiny_codegen = dlsym(TinyMPC_lib, "tiny_codegen")

function ccall_tiny_codegen!(nx, nu, N, Adyn_data, Bdyn_data, Q_data, R_data,
        x_min_data, x_max_data, u_min_data, u_max_data,
        rho_value, abs_pri_tol, abs_dual_tol, 
        max_iters, check_termination, gen_wrapper,
        tinympc_dir, output_dir)
    # - Use doubles here since tiny_codegen computes Riccati recursion which is numerically unstable
    # and using higher precision variables helps reduce that error.
    # - Variables are converted to tinytype (either float or double) when copying to the data_workspace
    # - All vectorized data is stored back into matrices in column major format!
    # For example, the vector Bdyn_data = {1,2,3,4,5,6} with nx = 3 and nu = 2 will be stored in Eigen 
    # as a matrix of size nx x nu = [[1, 4] 
    #                                [2, 5]
    #                                [3, 6]]
    @ccall $tiny_codegen(nx::Int64, nu::Int64, N::Int64,
        Adyn_data::Ptr{Cdouble}, Bdyn_data::Ptr{Cdouble}, Q_data::Ptr{Cdouble}, R_data::Ptr{Cdouble},
        x_min_data::Ptr{Cdouble}, x_max_data::Ptr{Cdouble}, u_min_data::Ptr{Cdouble}, u_max_data::Ptr{Cdouble},
        rho_value::Cdouble, abs_pri_tol::Cdouble, abs_dual_tol::Cdouble,
        max_iters::Int64, check_termination::Int64, gen_wrapper::Int64,
        tinympc_dir::Ptr{Cchar}, output_dir::Ptr{Cchar})::Cvoid
end

ccall_tiny_codegen! (generic function with 1 method)

In [47]:
Adyn_data
display(Array{Float64}(vec(A')))

16-element Vector{Float64}:
 1.0
 6.541101692727274e-5
 0.01
 2.18e-7
 0.0
 1.0010108975343306
 0.0
 0.01000336909090909
 0.0
 0.013084406770909093
 1.0
 6.541101692727274e-5
 0.0
 0.2022135591867769
 0.0
 1.0010108975343306

In [48]:
tinympc_dir = "/home/sam/Git/tinympc/TinyMPC/"
output_dir = "/generated_code"

# Adyn_data = Array{Float64}([1.0, 0.0, 0.0, 0.0, 0.01, 1.0, 0.0, 0.0, 2.2330083403300767e-5, 0.004466210576510177, 1.0002605176397052, 0.05210579005928538, 7.443037974683548e-8, 2.2330083403300767e-5, 0.01000086835443038, 1.0002605176397052])
# Bdyn_data = Array{Float64}([7.468368562730335e-5, 0.014936765390161838, 3.79763323185387e-5, 0.007595596218554721])
Adyn_data = Array{Float64}(vec(A'))
Bdyn_data = Array{Float64}(vec(B'))

# Q_data = Array{Float64}([10, 1, 10, 1])
# R_data = Array{Float64}([1])
# rho_value = 0.1;
Q_data = Array{Float64}(Q)
R_data = Array{Float64}(R)
rho_value = ρ;


# Set all min and max values to zero when generating code (doing this requires
# setting reasonable values before running mpc using the wrapper functions for
# the generated code)
x_min_data = Array{Float64}(zeros(nx * Nh))
x_max_data = Array{Float64}(zeros(nx * Nh))
u_min_data = Array{Float64}(zeros(nu * (Nh-1)))
u_max_data = Array{Float64}(zeros(nu * (Nh-1)))

abs_pri_tol = 1e-3;
abs_dual_tol = 1e-3;
max_iter = 100;
check_termination = 1; 
gen_wrapper = 1;

In [50]:
ccall_tiny_codegen!(nx, nu, Nh, Adyn_data, Bdyn_data, Q_data, R_data,
    x_min_data, x_max_data, u_min_data, u_max_data,
    rho_value, abs_pri_tol, abs_dual_tol,
    max_iter, check_termination, gen_wrapper,
    tinympc_dir, output_dir)

In [56]:
# Create wrappers for the generated C code

# create pointer to shared library
lib = dlopen(tinympc_dir * codegen_dir * "/build/tinympc/libtinympc.so")

# create pointers to tiny_wrapper functions
set_x = dlsym(lib, "set_x")
function ccall_set_x!(x, verbose)
    @ccall $set_x(x::Ptr{Cfloat}, verbose::Int64)::Cvoid 
end

set_x0 = dlsym(lib, "set_x0")
function ccall_set_x0!(x0, verbose)
    @ccall $set_x0(x0::Ptr{Cfloat}, verbose::Int64)::Cvoid 
end

set_xref = dlsym(lib, "set_xref")
function ccall_set_xref!(xref, verbose)
    @ccall $set_xref(xref::Ptr{Cfloat}, verbose::Int64)::Cvoid 
end

reset_dual_variables = dlsym(lib, "reset_dual_variables")
function ccall_reset_dual_variables!(verbose)
    @ccall $reset_dual_variables(verbose::Int64)::Cvoid
end

call_tiny_solve = dlsym(lib, "call_tiny_solve")
function ccall_tiny_solve!(verbose)
    @ccall $call_tiny_solve(verbose::Int64)::Cint
end

get_x = dlsym(lib, "get_x")
function ccall_get_x!(x_soln, verbose)
    @ccall $get_x(x_soln::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

get_u = dlsym(lib, "get_u")
function ccall_get_u!(u_soln, verbose)
    @ccall $get_u(u_soln::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

set_xmin = dlsym(lib, "set_xmin")
function ccall_set_xmin!(xmin, verbose)
    @ccall $set_xmin(xmin::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

set_xmax = dlsym(lib, "set_xmax")
function ccall_set_xmax!(xmax, verbose)
    @ccall $set_xmax(xmax::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

set_umin = dlsym(lib, "set_umin")
function ccall_set_umin!(umin, verbose)
    @ccall $set_umin(umin::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

set_umax = dlsym(lib, "set_umax")
function ccall_set_umax!(umax, verbose)
    @ccall $set_umax(umax::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

ccall_set_umax! (generic function with 1 method)

In [61]:
# # tests
# @show x0
# ccall_set_x!(x0)
# ccall_reset_dual_variables!()
# ccall_tiny_solve!()

xgoal = Array{Float32}(xgoal)
ccall_set_xref!(xgoal, 0)

xmin = [-100,-100,-100,-100]
xmax = [100,100,100,100]
xmin = Array{Float32}(xmin)
xmax = Array{Float32}(xmax)

umin = [-100]
umax = [100]
umin = Array{Float32}(umin)
umax = Array{Float32}(umax)


# x_soln = Array{Float32}(x_soln)
# ccall_print_x!(x_soln)
# @show Array{Float32}(x_soln)

1-element Vector{Float32}:
 100.0

In [62]:
# MPC loop

x_hist = [zeros(nx) for i = 1:Nsim] # store x values for sim
x_hist[1] = x0
u_hist = [zeros(nu) for i = 1:Nsim-1]
Nmax = 1
for k=1:Nsim-1
    # set initial condition and enforce float type
    current_x = Array{Float32}(vec(x_hist[k]))
    ccall_set_x0!(current_x, 0)

    ccall_set_xref!(xgoal, 0)

    ccall_set_xmin!(xmin, 0)
    ccall_set_xmax!(xmax, 0)

    ccall_set_umin!(umin, 0)
    ccall_set_umax!(umax, 0)

    # reset dual variables
    ccall_reset_dual_variables!(0)

    # solve
    ccall_tiny_solve!(0)

    # get u solution
    u_soln = Array{Float32}(zeros((nu,Nh-1)))
    ccall_get_u!(u_soln, 0)
    u_hist[k] = Array{Float32}(u_soln)[:,1]
    
    x_hist[k+1] = rk4(params,x_hist[k],u_hist[k],dt)
    @show Array{Float32}(x_hist[k+1])
end

In [64]:
# Simulate
display(animate_cartpole(x_hist[1:Nsim], dt))

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("pole" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "b" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x62], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "cart" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x63, 0x61, 0x72, 0x74], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "a" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x61], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), Set{Any}(), ip"127.0.0.1", 8705))